In [1]:
import dask

In [2]:
#import sys
#!{sys.executable} -m pip install dask_clickhouse

In [3]:
#!{sys.executable} -m pip install graphviz

In [4]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from datenspende.utils import query_ch_df, query_pg_df
import datetime
from datetime import date
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import re
import os
import glob
from functools import reduce

In [5]:
import ramda as R

from os import environ
from typing import TypedDict
from dask_clickhouse import read_clickhouse
from dask.dataframe import DataFrame as dd

class DBCredentials(TypedDict):
    user: str
    password: str
    database: str
    host: str
    port: int
    
def read_clickhouse_credentials_from_env() -> DBCredentials:
    return {
        "user": environ.get("CLICKHOUSE_USER"),
        "password": environ.get("CLICKHOUSE_PASSWORD"),
        "database": environ.get("CLICKHOUSE_DB"),
        "host": environ.get("CLICKHOUSE_HOST"),
        "port": int(environ.get("CLICKHOUSE_PORT")),
    }

def query_dd(query: str) -> dd:
    """
    Read query from clickhouse and return results as dask dataframe.
    
    Initialize a dask cluster before using this function via
    
    from dask_gateway import GatewayCluster
    cluster = GatewayCluster()
    """
    credentials = read_clickhouse_credentials_from_env()
    return read_clickhouse(
            query=query,
            connection_kwargs=credentials
    )

In [6]:
value_types = pd.read_csv('epoch_value_types.csv')
value_types = value_types.rename(columns={"id": "type"})

def add_s(st, end):
    
    if st == end:
        end += pd.Timedelta(seconds=1)
    return end

def clean_endv(st, end):
    if end == pd.Timestamp('1970-01-01 00:00:00'):
        end = st
    return end

In [7]:
def get_epoch(user_ids):
     
    ft = tuple(user_ids) 
    q =      """SELECT * FROM rocs.vital_data_epoch WHERE vital_data_epoch.customer IN {}""".format(ft)   
    
    
    us_data = query_dd(q)
    us_data = dd.merge(us_data, value_types, on='type')
    us_data = us_data.drop(columns=['type'])
    us_data = us_data.rename(columns={"code": "type"})
    us_data.startTimestamp = us_data.startTimestamp//1000
    us_data.endTimestamp = us_data.endTimestamp//1000
    us_data.startTimestamp = us_data.startTimestamp.apply(lambda x: datetime.datetime.fromtimestamp(x),meta=('startTimestamp', 'datetime64[ns]'))
    us_data.endTimestamp = us_data.endTimestamp.apply(lambda x: datetime.datetime.fromtimestamp(x),meta=('endTimestamp', 'datetime64[ns]'))
    us_data.endTimestamp  = us_data.apply(lambda x: clean_endv(x.startTimestamp, x.endTimestamp),axis=1, meta=(None, 'datetime64[ns]'))
    us_data.endTimestamp  = us_data.apply(lambda x: add_s(x.startTimestamp, x.endTimestamp),axis=1, meta=(None, 'datetime64[ns]'))
    us_data = us_data.rename(columns={"startTimestamp": "start", "endTimestamp": "end"})
    us_data = us_data.rename(columns={"customer": "id"})
    
    return us_data

In [13]:
# get age
def get_demo(user_ids):
    
    if isinstance(user_ids, int) or isinstance(user_ids, np.int64):
        formatter = f'({user_ids})'
    elif len(user_ids) == 1:
        formatter = f'({user_ids[0]})'
    else:
        formatter = tuple(user_ids) 
 
    query = f"""
    SELECT 
        user_id, salutation, birth_date, weight, height, creation_timestamp
    FROM 
        rocs.datenspende.users
    WHERE 
        users.user_id IN {formatter} 
    """ 

    ags = query_pg_df(query)
    ags.creation_timestamp = pd.to_datetime(ags['creation_timestamp'],unit='ms') 
    ags.creation_timestamp = ags.creation_timestamp.dt.date
    ags['age'] = np.floor((2023 + 1 / 12) - ags['birth_date'] + 2.5)
    
    qu = f"""
    select    
        a.user_id,
        a.created_at,
        a.question,
        a.element        
    from 
        rocs.datenspende.answers a
    where 
        a.user_id IN {formatter}
    AND
        a.question = 127    
    """
    sxs = query_pg_df(qu)
    sxs.created_at = pd.to_datetime(sxs['created_at'],unit='ms')
    sxs.created_at = sxs.created_at.dt.date
    
    
    if len(sxs) > 0:
        sex = 'female' if sxs['element'].values[0] == 773 else 'male'
    else: 
        sex = 'nd'
    if len(ags) > 0:
        age = ags['age'].values[0]
    else:
        age = 'nd'
    
    return sex, age

In [15]:
# this method takes all the users epoch data and puts it into bin_size_in_min bins of hr / steps data
# data explanation
# user_data is a dataframe for a single user, single source, HR + steps data
# columns
# id: user id
# longValue: HR in bpm
# doubleValue: steps [count]
# type: either "HeartRate" or "Steps"
# start: datetime start epoch
# end: datetime end epoch
# source: data source
# 
#      type source  doubleValue  longValue               id               start                 end
# HeartRate Fitbit          NaN       98.0       1234567890 2022-05-28 15:22:26 2022-05-28 15:41:13
#     Steps Fitbit       1526.0        NaN       1234567890 2022-05-28 15:22:26 2022-05-28 15:41:13
# HeartRate Fitbit          NaN      104.0       1234567890 2022-05-28 17:57:00 2022-05-28 19:31:47
#     Steps Fitbit       9269.0        NaN       1234567890 2022-05-28 17:57:00 2022-05-28 19:31:47
# HeartRate Fitbit          NaN      105.0       1234567890 2022-05-28 22:15:43 2022-05-28 22:40:30
def resample(us_data, bin_size_in_min):
    
        
    if len(us_data.index) > 50:
        
        user_data = us_data.copy()
        user_data = user_data[['id', 'doubleValue', 'longValue', 'booleanValue', 'start', 'end', 'source', 'type']]
        user_data = user_data.rename(
            columns={"longValue": "hr", "doubleValue": "steps", "booleanValue": "sleep"})
        user_data["duration"] = (user_data.end - user_data.start) / pd.Timedelta(
            "1 sec"
        )
        user_data = user_data.reset_index(drop=True )
        
        add_values = user_data[(user_data.duration > 1)&(user_data.type == 'Steps')]

        new_values = []
        for idx, row in add_values.iterrows():
            for i in np.arange(0, row.duration, 1):
                end_time = min(
                    row.end,
                    row.start
                    + pd.Timedelta("%d sec" % i)
                    + pd.Timedelta("%d sec" % 1),
                )
                new_duration = (
                    end_time - (row.start + pd.Timedelta("%d sec" % i))
                ) / pd.Timedelta("1 sec")
                new_values.append([
                    
                    row.id,
                    (row.steps / (row.duration / new_duration)),
                    row.hr,
                    row.sleep,
                    row.start + pd.Timedelta("%d sec" % i),
                    end_time,
                    row.source,
                    row.type,
                    new_duration,
                ])
        steps = user_data[(user_data.duration <= 1)&(user_data.type == 'Steps')].append(pd.DataFrame(data=new_values, columns=user_data.columns))
        steps = steps.sort_values(by='start')
        steps = steps.groupby(['start','type']).mean().reset_index()
        
        if 'hr' not in steps.columns:
            steps['hr'] = np.nan
        if 'sleep' not in steps.columns:
            steps['sleep'] = np.nan
            
        df = user_data[user_data.type != 'Steps'].append(steps)
        
        add_values = df[(df.duration > 60)]

        new_values = []
        add_values = add_values.fillna(np.nan)
        for idx, row in add_values.iterrows():
            for i in np.arange(0, row.duration, 60):
                end_time = min(
                    row.end,
                    row.start
                    + pd.Timedelta("%d sec" % i)
                    + pd.Timedelta("%d sec" % 60),
                )
                new_duration = (
                    end_time - (row.start + pd.Timedelta("%d sec" % i))
                ) / pd.Timedelta("1 sec")
                new_values.append([
                    
                    row.id,
                    (row.steps / (row.duration / new_duration)),
                    row.hr,
                    row.sleep,
                    row.start + pd.Timedelta("%d sec" % i),
                    end_time,
                    row.source,
                    row.type,
                    new_duration,
                ])
                
        df = df[df.duration <= 60].append(pd.DataFrame(data=new_values, columns=user_data.columns))
        df = df.sort_values(by='start')
        df = df.groupby(['start','type']).mean().reset_index()
        
        if 'hr' not in df.columns:
            df['hr'] = np.nan
        if 'sleep' not in df.columns:
            df['sleep'] = np.nan
        
        heartrate_bin = (
            df[df.type == "HeartRate"][["start", "hr"]]
            .set_index("start")
            .resample("%d Min" % bin_size_in_min)
            .mean()
            .reset_index()
        ).dropna(subset=["hr"])
        heartrate_bin["source"] = df.source.unique()[0]
        heartrate_bin["id"] = df.id.unique()[0]
      
        restheartrate_bin = (
            df[df.type == "HeartRateRestingHourly"][["start",  "hr"]]
            .set_index("start")
            .resample("%d Min" % bin_size_in_min)
            .mean()
            .reset_index()
        ).dropna(subset=["hr"])
       
        restheartrate_bin = restheartrate_bin.rename(columns={"hr": "rhr"})
  
        sleep_bin = (
            df[df.type == "SleepStateBinary"][["start",  "sleep"]]
            .set_index("start")
            .resample("%d Min" % bin_size_in_min)
            .mean()
            .reset_index()
        ).dropna(subset=["sleep"])

        steps_bin = (
            df[df.type == "Steps"][["start",  "steps"]]
            .set_index("start")
            .resample("%d Min" % bin_size_in_min)
            .sum()
            .reset_index()
        ).dropna(subset=["steps"])     

        data_frames = [heartrate_bin, restheartrate_bin, sleep_bin, steps_bin]
        df_lc = reduce(lambda  left,right: pd.merge(left,right,on=['start'],
                                                    how='outer'), data_frames)
        return df_lc
    return 

In [10]:
def phases(week):
    if week < 0:
        ph = 0
    elif (week >= 0 and week <= 4):
        ph = 1
    elif (week >= 5 and week <= 12):
        ph = 2
    elif week > 12:
        ph = 3
    return ph

In [11]:
pos_test = pd.read_csv('pos_testdate.csv',index_col=0)

In [ ]:
ud = get_epoch([146789])
sex, age = get_demo([146789])
 
if len(ud['source'].unique()) > 1:
    # if user has multiple devices, take device with max datapoints
    d_len = []
    for sour in ud['source'].unique():
        d_len.append(len(ud[ud['source'] == sour]))    
    ud = ud[ud['source'] == ud['source'].unique()[np.argmax(d_len)]]
    user = resample(ud, 15)    
else:
    user = resample(ud, 15)

/home/jovyan/datenspende-science/datenspende/utils/load_from_postgres.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)
/home/jovyan/datenspende-science/datenspende/utils/load_from_postgres.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


In [ ]:
 
        
    user['date'] = user.start.apply(lambda x: x.date())
    user['day_of_week'] = pd.to_datetime(user['date']).dt.dayofweek     
 
    user['weekend'] = user['day_of_week'].apply(lambda x: True if x >= 5 else False)

    user['dt'] = pd.to_datetime(pos_test['dt'][pos_test['user_id'].isin([us_id])].iloc[0])
    user['day_totest'] = pd.to_datetime(user['date']) - user['dt'] 
    user['week_totest'] = user['day_totest'].apply(lambda x: -(x.days// - 7))
    user['phase'] = user['week_totest'].apply(lambda x: phases(x))

    if sex == 'female':
        MHR = 206 - (0.88 * age)
    elif sex == 'male':
        MHR = 208 - (0.7 * age)
    else:
        
        MHR = np.nan

    user['% of MHR'] = (user['hr'])/ MHR * 100

    user.to_csv('user_epoch/'+sex+str(age)+str(us_id)+'.csv')

In [ ]:
us_data.head()